In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
def open_ai_completion(x):
    return f" {x['query']}\n" 

def execution(row, query_type):
    PATH = 'spider/database/'
    db_id = row['db_id']
    
    query = row[query_type]

    # Connect to the SQLite database file
    conn = sqlite3.connect(PATH + db_id + '/' + db_id + '.sqlite')

    # Create a cursor object
    cursor = conn.cursor()

    # Execute a SELECT statement
    try:
        cursor.execute(query)
    except:
        return "invalid SQL"

    # Fetch the results
    results = cursor.fetchall()

    # Loop through the results and print them
    # for result in results:
    #     print(result)

    # Close the cursor and connection
    cursor.close()
    conn.close()
    
    return results

# Fine tuned model on single tables
Execution accuracy is 82%

In [3]:
df = pd.read_csv('davinci-ft-mercator-2023-01-25-23-01-53-completions.csv',index_col=0)

df_spider = pd.read_json('spider/train_spider.json')
df_spider['open_ai_completion'] = df_spider.apply(lambda x: open_ai_completion(x), axis=1)


df = df.merge(df_spider[['db_id', 'open_ai_completion']], left_on='open_ai_completion', right_on='open_ai_completion')

df['open_ai_execution'] = df.apply(lambda x: execution(x, 'open_ai_completion'), axis=1)
df['model_response_execution'] = df.apply(lambda x: execution(x, 'model_response'), axis=1)
df['execution_accuracy'] = np.where(df['open_ai_execution'] == df['model_response_execution'], 1, 0)

df

,open_ai_prompt,open_ai_completion,model_response,db_id,open_ai_execution,model_response_execution,execution_accuracy
0,Schema: create table `catalog_contents` (`cata...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM CATAL...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
1,Schema: create table `catalog_contents` (`cata...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM CATAL...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
2,Schema: create table `catalog_contents` (`cata...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM Catal...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
3,Schema: create table `catalog_contents` (`cata...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM Catal...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
4,Schema: create table `attribute_definitions` (...,SELECT attribute_data_type FROM Attribute_Def...,SELECT attribute_data_type FROM attribute_def...,product_catalog,"[(Bool,)]","[(Bool,)]",1
...,...,...,...,...,...,...,...
985,Schema: create table airports ( apid integer...,"SELECT avg(elevation) , country FROM airport...","SELECT country , avg(elevation) FROM airport...",flight_4,"[(3732.4, Afghanistan), (281.4, Albania), (144...","[(Afghanistan, 3732.4), (Albania, 281.4), (Alg...",0
986,Schema: create table airports ( apid integer...,SELECT city FROM airports GROUP BY city HAVIN...,SELECT city FROM airports GROUP BY city HAVIN...,flight_4,"[(Abilene,), (Adana,), (Addis Ababa,), (Adelai...","[(Abilene,), (Adana,), (Addis Ababa,), (Adelai...",1
987,Schema: create table airports ( apid integer...,SELECT city FROM airports GROUP BY city HAVIN...,SELECT city FROM airports GROUP BY city HAVIN...,flight_4,"[(Abilene,), (Adana,), (Addis Ababa,), (Adelai...","[(Abilene,), (Adana,), (Addis Ababa,), (Adelai...",1
988,Schema: create table airports ( apid integer...,SELECT city FROM airports GROUP BY city HAVIN...,SELECT city FROM airports GROUP BY city HAVIN...,flight_4,"[(Abilene,), (Adana,), (Addis Ababa,), (Adelai...","[(Abilene,), (Adana,), (Addis Ababa,), (Adelai...",1


In [9]:
df[df['open_ai_completion'].str.contains('JOIN')]

,open_ai_prompt,open_ai_completion,model_response,db_id,open_ai_execution,model_response_execution,execution_accuracy


In [4]:
df['execution_accuracy'].mean()

0.8181818181818182

In [5]:
df[df['model_response_execution'] == 'invalid SQL']['model_response']

82      SELECT sum ,  avg(settlement_amount) FROM Set...
83      SELECT sum ,  avg(settlement_amount) FROM Set...
231     SELECT dependent_name FROM dependent WHERE re...
352     SELECT hispanic_percent FROM city WHERE black...
353     SELECT hispanic_percent FROM city WHERE black...
406     SELECT count(DISTINCT affected_region) FROM a...
407     SELECT count(DISTINCT affected_region) FROM a...
471     SELECT format FROM files GROUP BY format ORDE...
472     SELECT format FROM files GROUP BY format ORDE...
479     SELECT language ,  avg(rating) FROM song GROU...
480     SELECT language ,  avg(rating) FROM song GROU...
481     SELECT language ,  avg(rating) FROM song GROU...
482     SELECT language ,  avg(rating) FROM song GROU...
483     SELECT format ,  count(DISTINCT artist_name) ...
484     SELECT format ,  count(DISTINCT artist_name) ...
485     SELECT count(*) ,  format FROM Files GROUP BY...
486     SELECT count(*) ,  format FROM Files GROUP BY...
509     SELECT song_name FROM S

In [8]:
print(df.loc[82]['open_ai_prompt'])
print(df.loc[82]['model_response'])

Schema: create table settlements (settlement_id integer not null,claim_id integer,effective_date date,settlement_amount real,primary key (settlement_id),unique (settlement_id),foreign key (claim_id);
Question: Return the sum and average of all settlement amounts.

###


 SELECT sum ,  avg(settlement_amount) FROM Settlements


In [30]:
# 30 swaps column order
# 40 needs to see the table contents to know how countries are abbreviated
# 60 swaps the column order
i = 80
print(df[df['execution_accuracy'] == 0]['open_ai_prompt'].iloc[i])
print(df[df['execution_accuracy'] == 0]['open_ai_completion'].iloc[i])
print(df[df['execution_accuracy'] == 0]['model_response'].iloc[i])

Schema: create table files(	f_id number(10) not null,	artist_name varchar2(50),	file_size varchar2(20),	duration varchar2(20),	formats varchar2(20),	primary key(f_id),	foreign key(artist_name);
Question: For each file format, return the number of artists who released songs in that format.

###


 SELECT count(*) ,  formats FROM files GROUP BY formats

 SELECT format ,  count(DISTINCT artist_name) FROM Files GROUP BY format


# Fine tuned model on multiple tables
Execution accuracy is 51%

In [10]:
df_multi = pd.read_csv('df_test_davinci_ft-mercator-2023-01-27-10-13-07.csv',index_col=0).fillna('')

df_spider = pd.read_json('spider/train_spider.json')
df_spider['open_ai_completion'] = df_spider.apply(lambda x: open_ai_completion(x), axis=1)


df_multi = df_multi.merge(df_spider[['db_id', 'open_ai_completion']], 
                          left_on='open_ai_completion', 
                          right_on='open_ai_completion')

df_multi['open_ai_execution'] = df_multi.apply(lambda x: execution(x, 'open_ai_completion'), axis=1)
df_multi['model_response_execution'] = df_multi.apply(lambda x: execution(x, 'model_response'), axis=1)
df_multi['execution_accuracy'] = np.where(df_multi['open_ai_execution'] == df_multi['model_response_execution'], 1, 0)

df_multi

,open_ai_prompt,open_ai_completion,model_response,db_id,open_ai_execution,model_response_execution,execution_accuracy
0,Schema: CREATE TABLE `Attribute_Definitions` (...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM catal...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
1,Schema: CREATE TABLE `Attribute_Definitions` (...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM catal...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
2,Schema: CREATE TABLE `Attribute_Definitions` (...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM catal...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
3,Schema: CREATE TABLE `Attribute_Definitions` (...,SELECT distinct(catalog_entry_name) FROM cata...,SELECT DISTINCT catalog_entry_name FROM catal...,product_catalog,"[(Cola,), (Root beer,), (Cream Soda,), (Carbon...","[(Cola,), (Root beer,), (Cream Soda,), (Carbon...",1
4,Schema: CREATE TABLE `Attribute_Definitions` (...,SELECT attribute_data_type FROM Attribute_Def...,SELECT attribute_data_type FROM attribute_def...,product_catalog,"[(Bool,)]","[(Bool,)]",1
...,...,...,...,...,...,...,...
1931,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.name FROM airports AS T1 JOIN route...,flight_4,"[(Beijing Capital International Airport,)]",[],0
1932,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.dst_ap FROM routes AS T1 JOIN airpo...,flight_4,"[(Beijing Capital International Airport,)]",invalid SQL,0
1933,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.dst_ap FROM routes AS T1 JOIN airpo...,flight_4,"[(Beijing Capital International Airport,)]",invalid SQL,0
1934,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.name FROM airports AS T1 JOIN route...,flight_4,"[(Beijing Capital International Airport,)]",[],0


In [11]:
df_multi[df_multi['open_ai_completion'].str.contains('JOIN')]

,open_ai_prompt,open_ai_completion,model_response,db_id,open_ai_execution,model_response_execution,execution_accuracy
24,Schema: CREATE TABLE `Attribute_Definitions` (...,"SELECT t1.catalog_name , t1.date_of_publicat...","SELECT catalog_name , date_of_publication FR...",product_catalog,"[(Coffee Bean, 2012-04-13 06:37:09), (Jam, 201...",invalid SQL,0
25,Schema: CREATE TABLE `Attribute_Definitions` (...,"SELECT t1.catalog_name , t1.date_of_publicat...","SELECT catalog_name , date_of_publication FR...",product_catalog,"[(Coffee Bean, 2012-04-13 06:37:09), (Jam, 201...",invalid SQL,0
26,Schema: CREATE TABLE `Attribute_Definitions` (...,"SELECT t1.catalog_name , t1.date_of_publicat...","SELECT T1.catalog_name , T1.date_of_publicat...",product_catalog,"[(Coffee Bean, 2012-04-13 06:37:09), (Jam, 201...","[(Coffee Bean, 2012-04-13 06:37:09), (Jam, 201...",1
27,Schema: CREATE TABLE `Attribute_Definitions` (...,"SELECT t1.catalog_name , t1.date_of_publicat...","SELECT T1.catalog_name , T1.date_of_publicat...",product_catalog,"[(Coffee Bean, 2012-04-13 06:37:09), (Jam, 201...","[(Coffee Bean, 2012-04-13 06:37:09), (Jam, 201...",1
28,Schema: CREATE TABLE `Attribute_Definitions` (...,SELECT t1.catalog_entry_name FROM Catalog_Con...,SELECT T1.catalog_entry_name FROM Catalog_Con...,product_catalog,"[(Wanglaoji,), (Diet Pepsi,), (Diet Mountain D...","[(Wanglaoji,)]",0
...,...,...,...,...,...,...,...
1931,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.name FROM airports AS T1 JOIN route...,flight_4,"[(Beijing Capital International Airport,)]",[],0
1932,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.dst_ap FROM routes AS T1 JOIN airpo...,flight_4,"[(Beijing Capital International Airport,)]",invalid SQL,0
1933,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.dst_ap FROM routes AS T1 JOIN airpo...,flight_4,"[(Beijing Capital International Airport,)]",invalid SQL,0
1934,Schema: CREATE TABLE routes ( rid integer PR...,SELECT T1.name FROM airports AS T1 JOIN route...,SELECT T1.name FROM airports AS T1 JOIN route...,flight_4,"[(Beijing Capital International Airport,)]",[],0


In [12]:
df_multi['execution_accuracy'].mean()

0.5077479338842975

In [20]:
i = 20
print(df_multi[df_multi['execution_accuracy'] == 0]['open_ai_completion'].iloc[i])
print(df_multi[df_multi['execution_accuracy'] == 0]['open_ai_execution'].iloc[i])
print(df_multi[df_multi['execution_accuracy'] == 0]['model_response'].iloc[i])
print(df_multi[df_multi['execution_accuracy'] == 0]['model_response_execution'].iloc[i])

 SELECT catalog_entry_name FROM catalog_contents ORDER BY height DESC LIMIT 1

[('Cola',)]
 SELECT t1.catalog_entry_name FROM catalog_contents AS t1 JOIN catalog_contents_additional_attributes AS t2 ON t1.catalog_entry_id  =  t2.catalog_entry_id WHERE t2.attribute_value  =  (SELECT max(attribute_value) FROM catalog_contents_additional_attributes) ORDER BY t2.attribute_value
[('Ginger Beer',), ('Cola',), ('Ramune',), ('Wanglaoji',), ('Diet Pepsi',), ('Root beer',), ('Tizer',)]
